In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

In [2]:
data = pd.read_csv("SOUND_CLASSIFICATION_DATASET/training1.csv")

In [11]:
data.head()

,NAME,LABEL
0,1,1
1,2,1
2,3,1
3,4,0
4,5,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1584 entries, 0 to 1583
Data columns (total 2 columns):
NAME     1584 non-null int64
LABEL    1584 non-null int64
dtypes: int64(2)
memory usage: 24.8 KB


In [5]:
from librosa import display
import librosa

In [10]:
# PREPROCESSING 
x_train=[]
y_train=[]
path="SOUND_CLASSIFICATION_DATASET/TRAIN_SET/"
for i in tqdm(range(len(data))):
    file=str(data.iloc[i]["NAME"])+".wav"
    label=data.iloc[i]["LABEL"]
    filename=path+file
    y,sr=librosa.load(filename)
    mfccs = np.mean(librosa.feature.mfcc(y, sr, n_mfcc=40).T,axis=0)
    melspectrogram = np.mean(librosa.feature.melspectrogram(y=y, sr=sr, n_mels=40,fmax=8000).T,axis=0)
    chroma_stft=np.mean(librosa.feature.chroma_stft(y=y, sr=sr,n_chroma=40).T,axis=0)
    chroma_cq = np.mean(librosa.feature.chroma_cqt(y=y, sr=sr,n_chroma=40).T,axis=0)
    chroma_cens = np.mean(librosa.feature.chroma_cens(y=y, sr=sr,n_chroma=40).T,axis=0)
    features=np.reshape(np.vstack((mfccs,melspectrogram,chroma_stft,chroma_cq,chroma_cens)),(40,5))
    x_train.append(features)
    y_train.append(label)

  8%|██████▋                                                                        | 133/1584 [01:25<16:08,  1.50it/s]C:\Users\lenovo\Anaconda3\envs\tensorflow\lib\site-packages\librosa\core\pitch.py:146: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')
100%|██████████████████████████████████████████████████████████████████████████████| 1584/1584 [15:51<00:00,  1.66it/s]


In [12]:
dataset = pd.read_csv("SOUND_CLASSIFICATION_DATASET/testing1.csv")

In [13]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 2 columns):
NAME     56 non-null int64
LABEL    56 non-null int64
dtypes: int64(2)
memory usage: 976.0 bytes


In [14]:
# PREPROCESSING TEST SET
x_test=[]
y_test=[]
path="SOUND_CLASSIFICATION_DATASET/TEST_SET/"
for i in tqdm(range(len(dataset))):
    file=str(dataset.iloc[i]["NAME"])+".wav"
    label=dataset.iloc[i]["LABEL"]
    filename=path+file
    y,sr=librosa.load(filename)
    mfccs = np.mean(librosa.feature.mfcc(y, sr, n_mfcc=40).T,axis=0)
    melspectrogram = np.mean(librosa.feature.melspectrogram(y=y, sr=sr, n_mels=40,fmax=8000).T,axis=0)
    chroma_stft=np.mean(librosa.feature.chroma_stft(y=y, sr=sr,n_chroma=40).T,axis=0)
    chroma_cq = np.mean(librosa.feature.chroma_cqt(y=y, sr=sr,n_chroma=40).T,axis=0)
    chroma_cens = np.mean(librosa.feature.chroma_cens(y=y, sr=sr,n_chroma=40).T,axis=0)
    features=np.reshape(np.vstack((mfccs,melspectrogram,chroma_stft,chroma_cq,chroma_cens)),(40,5))
    x_test.append(features)
    y_test.append(label)

100%|██████████████████████████████████████████████████████████████████████████████████| 56/56 [00:35<00:00,  1.59it/s]


In [15]:
# CONVERTING TO ARRAY
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)
x_test = np.asarray(x_test)
y_test = np.asarray(y_test)

In [16]:
x_train.shape , y_train.shape , x_test.shape , y_test.shape

((1584, 40, 5), (1584,), (56, 40, 5), (56,))

In [17]:
y_train = y_train.reshape((1584,1))
y_test = y_test.reshape((56,1))

In [18]:
x_test = x_test.reshape((56,40,5,1))

In [19]:
x_test.shape

(56, 40, 5, 1)

In [20]:
#converting to one hot
from keras.utils.np_utils import to_categorical
y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)
y_train.shape,y_test.shape

Using TensorFlow backend.


((1584, 2), (56, 2))

In [21]:
y_train[1583]

array([0., 1.], dtype=float32)

In [22]:
x_train = x_train.reshape((1584,40,5,1))

In [23]:
x_train.shape

(1584, 40, 5, 1)

In [24]:
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Dropout

In [25]:
model = Sequential()

In [26]:
#adding layers and forming the model
model.add(Conv2D(64,kernel_size=5,strides=1,padding="Same",activation="relu",input_shape=(40,5,1)))
model.add(MaxPooling2D(padding="same"))

model.add(Conv2D(128,kernel_size=5,strides=1,padding="same",activation="relu"))
model.add(MaxPooling2D(padding="same"))
model.add(Dropout(0.3))

model.add(Flatten())

model.add(Dense(256,activation="relu"))
model.add(Dropout(0.3))

model.add(Dense(512,activation="relu"))
model.add(Dropout(0.3))

model.add(Dense(2,activation="softmax"))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [27]:
#compiling
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [28]:
# Training the model
model.fit(x_train,y_train,batch_size=32,epochs=50,validation_data=(x_test,y_test))

Instructions for updating:
Use tf.cast instead.
Train on 1584 samples, validate on 56 samples
Epoch 1/50
1584/1584 [==============================] - ETA: 3:58 - loss: 2.1688 - acc: 0.406 - ETA: 2:01 - loss: 4.6602 - acc: 0.375 - ETA: 1:21 - loss: 4.2112 - acc: 0.364 - ETA: 1:02 - loss: 3.8800 - acc: 0.390 - ETA: 49s - loss: 3.4624 - acc: 0.443 - ETA: 42s - loss: 3.1500 - acc: 0.46 - ETA: 36s - loss: 2.8436 - acc: 0.47 - ETA: 31s - loss: 2.6604 - acc: 0.50 - ETA: 28s - loss: 2.6087 - acc: 0.50 - ETA: 25s - loss: 2.4431 - acc: 0.51 - ETA: 23s - loss: 2.3311 - acc: 0.50 - ETA: 21s - loss: 2.2462 - acc: 0.49 - ETA: 19s - loss: 2.1663 - acc: 0.50 - ETA: 18s - loss: 2.0784 - acc: 0.49 - ETA: 16s - loss: 1.9936 - acc: 0.50 - ETA: 15s - loss: 1.9358 - acc: 0.49 - ETA: 14s - loss: 1.8615 - acc: 0.49 - ETA: 13s - loss: 1.8028 - acc: 0.50 - ETA: 12s - loss: 1.7374 - acc: 0.51 - ETA: 11s - loss: 1.7021 - acc: 0.51 - ETA: 11s - loss: 1.6685 - acc: 0.50 - ETA: 10s - loss: 1.6373 - acc: 0.51 - ETA: 

1584/1584 [==============================] - ETA: 7s - loss: 0.7296 - acc: 0.875 - ETA: 7s - loss: 0.5648 - acc: 0.828 - ETA: 7s - loss: 0.4342 - acc: 0.875 - ETA: 7s - loss: 0.4019 - acc: 0.882 - ETA: 7s - loss: 0.3587 - acc: 0.893 - ETA: 7s - loss: 0.3870 - acc: 0.880 - ETA: 7s - loss: 0.3859 - acc: 0.875 - ETA: 6s - loss: 0.3680 - acc: 0.882 - ETA: 6s - loss: 0.3464 - acc: 0.888 - ETA: 6s - loss: 0.3235 - acc: 0.896 - ETA: 6s - loss: 0.3173 - acc: 0.894 - ETA: 6s - loss: 0.3122 - acc: 0.895 - ETA: 6s - loss: 0.3098 - acc: 0.896 - ETA: 5s - loss: 0.3402 - acc: 0.886 - ETA: 5s - loss: 0.3397 - acc: 0.883 - ETA: 5s - loss: 0.3448 - acc: 0.882 - ETA: 5s - loss: 0.3391 - acc: 0.884 - ETA: 5s - loss: 0.3385 - acc: 0.885 - ETA: 4s - loss: 0.3323 - acc: 0.888 - ETA: 4s - loss: 0.3298 - acc: 0.887 - ETA: 4s - loss: 0.3265 - acc: 0.885 - ETA: 4s - loss: 0.3275 - acc: 0.887 - ETA: 4s - loss: 0.3296 - acc: 0.887 - ETA: 4s - loss: 0.3240 - acc: 0.889 - ETA: 4s - loss: 0.3261 - acc: 0.888 - ETA: 

1584/1584 [==============================] - ETA: 9s - loss: 0.1042 - acc: 0.968 - ETA: 8s - loss: 0.1023 - acc: 0.968 - ETA: 7s - loss: 0.1169 - acc: 0.958 - ETA: 7s - loss: 0.1213 - acc: 0.960 - ETA: 7s - loss: 0.1406 - acc: 0.950 - ETA: 7s - loss: 0.1423 - acc: 0.942 - ETA: 7s - loss: 0.1251 - acc: 0.950 - ETA: 6s - loss: 0.1385 - acc: 0.941 - ETA: 6s - loss: 0.1359 - acc: 0.937 - ETA: 6s - loss: 0.1470 - acc: 0.934 - ETA: 6s - loss: 0.1613 - acc: 0.934 - ETA: 6s - loss: 0.1659 - acc: 0.929 - ETA: 5s - loss: 0.1649 - acc: 0.932 - ETA: 5s - loss: 0.1658 - acc: 0.935 - ETA: 5s - loss: 0.1587 - acc: 0.939 - ETA: 5s - loss: 0.1604 - acc: 0.941 - ETA: 5s - loss: 0.1598 - acc: 0.941 - ETA: 5s - loss: 0.1613 - acc: 0.939 - ETA: 4s - loss: 0.1634 - acc: 0.935 - ETA: 4s - loss: 0.1595 - acc: 0.937 - ETA: 4s - loss: 0.1549 - acc: 0.940 - ETA: 4s - loss: 0.1550 - acc: 0.938 - ETA: 4s - loss: 0.1648 - acc: 0.932 - ETA: 4s - loss: 0.1699 - acc: 0.929 - ETA: 3s - loss: 0.1694 - acc: 0.931 - ETA: 

1584/1584 [==============================] - ETA: 6s - loss: 0.1640 - acc: 0.906 - ETA: 7s - loss: 0.1478 - acc: 0.921 - ETA: 7s - loss: 0.1294 - acc: 0.937 - ETA: 7s - loss: 0.1214 - acc: 0.953 - ETA: 6s - loss: 0.1168 - acc: 0.956 - ETA: 6s - loss: 0.1434 - acc: 0.942 - ETA: 6s - loss: 0.1366 - acc: 0.946 - ETA: 6s - loss: 0.1526 - acc: 0.937 - ETA: 6s - loss: 0.1685 - acc: 0.937 - ETA: 6s - loss: 0.2218 - acc: 0.931 - ETA: 6s - loss: 0.2148 - acc: 0.931 - ETA: 5s - loss: 0.2083 - acc: 0.932 - ETA: 5s - loss: 0.2008 - acc: 0.935 - ETA: 5s - loss: 0.2108 - acc: 0.930 - ETA: 5s - loss: 0.2063 - acc: 0.933 - ETA: 5s - loss: 0.2064 - acc: 0.933 - ETA: 5s - loss: 0.2084 - acc: 0.935 - ETA: 4s - loss: 0.2106 - acc: 0.932 - ETA: 4s - loss: 0.2071 - acc: 0.932 - ETA: 4s - loss: 0.2048 - acc: 0.932 - ETA: 4s - loss: 0.2014 - acc: 0.933 - ETA: 4s - loss: 0.1982 - acc: 0.933 - ETA: 4s - loss: 0.1923 - acc: 0.934 - ETA: 3s - loss: 0.1910 - acc: 0.932 - ETA: 3s - loss: 0.1908 - acc: 0.931 - ETA: 

1584/1584 [==============================] - ETA: 9s - loss: 0.1827 - acc: 0.906 - ETA: 9s - loss: 0.1676 - acc: 0.921 - ETA: 8s - loss: 0.1212 - acc: 0.947 - ETA: 8s - loss: 0.1318 - acc: 0.937 - ETA: 8s - loss: 0.1220 - acc: 0.937 - ETA: 7s - loss: 0.1052 - acc: 0.947 - ETA: 7s - loss: 0.0930 - acc: 0.955 - ETA: 7s - loss: 0.0932 - acc: 0.953 - ETA: 7s - loss: 0.1011 - acc: 0.954 - ETA: 7s - loss: 0.1119 - acc: 0.953 - ETA: 7s - loss: 0.1104 - acc: 0.951 - ETA: 8s - loss: 0.1207 - acc: 0.945 - ETA: 7s - loss: 0.1172 - acc: 0.947 - ETA: 7s - loss: 0.1207 - acc: 0.942 - ETA: 7s - loss: 0.1207 - acc: 0.941 - ETA: 7s - loss: 0.1225 - acc: 0.941 - ETA: 6s - loss: 0.1298 - acc: 0.939 - ETA: 6s - loss: 0.1389 - acc: 0.935 - ETA: 6s - loss: 0.1383 - acc: 0.934 - ETA: 5s - loss: 0.1374 - acc: 0.934 - ETA: 5s - loss: 0.1368 - acc: 0.933 - ETA: 5s - loss: 0.1356 - acc: 0.933 - ETA: 5s - loss: 0.1327 - acc: 0.934 - ETA: 5s - loss: 0.1422 - acc: 0.931 - ETA: 4s - loss: 0.1382 - acc: 0.933 - ETA: 

...

1584/1584 [==============================] - ETA: 7s - loss: 0.0399 - acc: 1.000 - ETA: 7s - loss: 0.0410 - acc: 1.000 - ETA: 7s - loss: 0.0452 - acc: 0.989 - ETA: 7s - loss: 0.0421 - acc: 0.992 - ETA: 7s - loss: 0.0553 - acc: 0.987 - ETA: 7s - loss: 0.0481 - acc: 0.989 - ETA: 7s - loss: 0.0466 - acc: 0.991 - ETA: 7s - loss: 0.0462 - acc: 0.992 - ETA: 6s - loss: 0.0429 - acc: 0.993 - ETA: 6s - loss: 0.0505 - acc: 0.990 - ETA: 6s - loss: 0.0473 - acc: 0.991 - ETA: 6s - loss: 0.0455 - acc: 0.992 - ETA: 6s - loss: 0.0442 - acc: 0.992 - ETA: 6s - loss: 0.0426 - acc: 0.993 - ETA: 5s - loss: 0.0399 - acc: 0.993 - ETA: 5s - loss: 0.0385 - acc: 0.994 - ETA: 5s - loss: 0.0370 - acc: 0.994 - ETA: 5s - loss: 0.0352 - acc: 0.994 - ETA: 5s - loss: 0.0388 - acc: 0.991 - ETA: 4s - loss: 0.0369 - acc: 0.992 - ETA: 4s - loss: 0.0364 - acc: 0.991 - ETA: 4s - loss: 0.0356 - acc: 0.991 - ETA: 4s - loss: 0.0348 - acc: 0.991 - ETA: 4s - loss: 0.0360 - acc: 0.990 - ETA: 4s - loss: 0.0389 - acc: 0.990 - ETA: 

1584/1584 [==============================] - ETA: 8s - loss: 0.0455 - acc: 0.968 - ETA: 8s - loss: 0.0698 - acc: 0.984 - ETA: 7s - loss: 0.0556 - acc: 0.989 - ETA: 7s - loss: 0.0435 - acc: 0.992 - ETA: 7s - loss: 0.0386 - acc: 0.993 - ETA: 7s - loss: 0.0404 - acc: 0.994 - ETA: 7s - loss: 0.0392 - acc: 0.995 - ETA: 7s - loss: 0.0525 - acc: 0.984 - ETA: 7s - loss: 0.0475 - acc: 0.986 - ETA: 6s - loss: 0.0509 - acc: 0.984 - ETA: 6s - loss: 0.0631 - acc: 0.980 - ETA: 6s - loss: 0.0694 - acc: 0.976 - ETA: 6s - loss: 0.0655 - acc: 0.978 - ETA: 6s - loss: 0.0626 - acc: 0.979 - ETA: 5s - loss: 0.0614 - acc: 0.981 - ETA: 5s - loss: 0.0589 - acc: 0.982 - ETA: 5s - loss: 0.0572 - acc: 0.983 - ETA: 5s - loss: 0.0551 - acc: 0.984 - ETA: 5s - loss: 0.0579 - acc: 0.981 - ETA: 5s - loss: 0.0590 - acc: 0.979 - ETA: 4s - loss: 0.0567 - acc: 0.980 - ETA: 4s - loss: 0.0619 - acc: 0.978 - ETA: 4s - loss: 0.0631 - acc: 0.976 - ETA: 4s - loss: 0.0622 - acc: 0.976 - ETA: 4s - loss: 0.0611 - acc: 0.976 - ETA: 

1584/1584 [==============================] - ETA: 8s - loss: 0.0062 - acc: 1.000 - ETA: 7s - loss: 0.0201 - acc: 1.000 - ETA: 7s - loss: 0.0165 - acc: 1.000 - ETA: 7s - loss: 0.0220 - acc: 0.992 - ETA: 7s - loss: 0.0352 - acc: 0.981 - ETA: 7s - loss: 0.0646 - acc: 0.974 - ETA: 7s - loss: 0.0656 - acc: 0.968 - ETA: 7s - loss: 0.0697 - acc: 0.964 - ETA: 6s - loss: 0.0717 - acc: 0.961 - ETA: 6s - loss: 0.0658 - acc: 0.965 - ETA: 6s - loss: 0.0638 - acc: 0.965 - ETA: 6s - loss: 0.0612 - acc: 0.968 - ETA: 6s - loss: 0.0596 - acc: 0.968 - ETA: 6s - loss: 0.0575 - acc: 0.971 - ETA: 5s - loss: 0.0648 - acc: 0.968 - ETA: 5s - loss: 0.0620 - acc: 0.970 - ETA: 5s - loss: 0.0587 - acc: 0.972 - ETA: 5s - loss: 0.0558 - acc: 0.974 - ETA: 5s - loss: 0.0540 - acc: 0.975 - ETA: 5s - loss: 0.0519 - acc: 0.976 - ETA: 4s - loss: 0.0500 - acc: 0.977 - ETA: 4s - loss: 0.0541 - acc: 0.977 - ETA: 4s - loss: 0.0527 - acc: 0.978 - ETA: 4s - loss: 0.0531 - acc: 0.976 - ETA: 4s - loss: 0.0653 - acc: 0.975 - ETA: 

1584/1584 [==============================] - ETA: 7s - loss: 0.0039 - acc: 1.000 - ETA: 7s - loss: 0.0104 - acc: 1.000 - ETA: 8s - loss: 0.0539 - acc: 0.989 - ETA: 7s - loss: 0.0479 - acc: 0.992 - ETA: 7s - loss: 0.1015 - acc: 0.987 - ETA: 7s - loss: 0.0864 - acc: 0.989 - ETA: 7s - loss: 0.0812 - acc: 0.991 - ETA: 7s - loss: 0.0743 - acc: 0.988 - ETA: 7s - loss: 0.0674 - acc: 0.989 - ETA: 6s - loss: 0.0721 - acc: 0.987 - ETA: 6s - loss: 0.0864 - acc: 0.983 - ETA: 7s - loss: 0.0940 - acc: 0.979 - ETA: 7s - loss: 0.0900 - acc: 0.980 - ETA: 7s - loss: 0.0863 - acc: 0.979 - ETA: 6s - loss: 0.0831 - acc: 0.979 - ETA: 6s - loss: 0.0796 - acc: 0.980 - ETA: 6s - loss: 0.0827 - acc: 0.979 - ETA: 6s - loss: 0.0806 - acc: 0.979 - ETA: 6s - loss: 0.0781 - acc: 0.980 - ETA: 5s - loss: 0.0769 - acc: 0.979 - ETA: 5s - loss: 0.0740 - acc: 0.980 - ETA: 5s - loss: 0.0734 - acc: 0.980 - ETA: 5s - loss: 0.0723 - acc: 0.979 - ETA: 5s - loss: 0.0843 - acc: 0.976 - ETA: 4s - loss: 0.0813 - acc: 0.977 - ETA: 

1584/1584 [==============================] - ETA: 8s - loss: 0.0248 - acc: 1.000 - ETA: 8s - loss: 0.0124 - acc: 1.000 - ETA: 8s - loss: 0.0159 - acc: 1.000 - ETA: 7s - loss: 0.0121 - acc: 1.000 - ETA: 7s - loss: 0.0142 - acc: 1.000 - ETA: 7s - loss: 0.0122 - acc: 1.000 - ETA: 7s - loss: 0.0215 - acc: 0.995 - ETA: 7s - loss: 0.0351 - acc: 0.984 - ETA: 7s - loss: 0.0312 - acc: 0.986 - ETA: 6s - loss: 0.0284 - acc: 0.987 - ETA: 6s - loss: 0.0272 - acc: 0.988 - ETA: 6s - loss: 0.0302 - acc: 0.987 - ETA: 6s - loss: 0.0282 - acc: 0.988 - ETA: 6s - loss: 0.0276 - acc: 0.988 - ETA: 5s - loss: 0.0258 - acc: 0.989 - ETA: 5s - loss: 0.0244 - acc: 0.990 - ETA: 5s - loss: 0.0232 - acc: 0.990 - ETA: 5s - loss: 0.0292 - acc: 0.989 - ETA: 5s - loss: 0.0304 - acc: 0.988 - ETA: 4s - loss: 0.0362 - acc: 0.987 - ETA: 4s - loss: 0.0359 - acc: 0.988 - ETA: 4s - loss: 0.0347 - acc: 0.988 - ETA: 4s - loss: 0.0361 - acc: 0.987 - ETA: 4s - loss: 0.0359 - acc: 0.987 - ETA: 4s - loss: 0.0349 - acc: 0.987 - ETA: 

In [88]:
mode.get_weights()

[array([[[[ 0.01125067, -0.03761831,  0.00506061, ...,  0.01135404,
           -0.01163099,  0.01054627]],
 
         [[ 0.000873  , -0.01929228, -0.00669033, ...,  0.02965062,
            0.00422322, -0.02420651]],
 
         [[-0.04785929, -0.04232047, -0.01231945, ...,  0.01878584,
            0.09917814, -0.0047835 ]],
 
         [[-0.12049931, -0.03172345,  0.00635783, ..., -0.07323691,
           -0.06069348,  0.01804657]],
 
         [[-0.01670641, -0.03143491,  0.01803452, ..., -0.01929908,
           -0.19950385,  0.08126143]]],
 
 
        [[[ 0.02750266,  0.06977442, -0.0290975 , ...,  0.03676389,
            0.00761043, -0.0290525 ]],
 
         [[ 0.00993804, -0.00144175, -0.14186163, ..., -0.09806665,
            0.01136503, -0.06294262]],
 
         [[ 0.0057756 , -0.01529799, -0.04213442, ..., -0.04734529,
            0.00867265, -0.05124036]],
 
         [[ 0.00039044,  0.08845065,  0.0944014 , ..., -0.14531694,
           -0.03648807, -0.05597885]],
 
         [[ 0.04

In [29]:
score_train = model.evaluate(x_train,y_train)
score_test = model.evaluate(x_test,y_test)

56/56 [==============================] - ETA:  - 0s 1ms/step


In [30]:
print("Accuracy on training set : ", score_train[1])
print("Accuracy on testing set : ", score_test[1])

Accuracy on training set :  0.9924242424242424
Accuracy on testing set :  0.9642857142857143


In [31]:
model.save("speech_model_correct.h5")